In [ ]:
import numpy
import tensorflow as tf
import tensorflow.keras
from keras.utils import np_utils
from tensorflow.keras import Sequential
from music21 import converter,stream ,instrument,chord,note
import glob
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense,Dropout,LSTM,Activation

In [ ]:
notes = []
for file in glob.glob("Input.mid"):
    midi = converter.parse(file)  #parsing a midi file
    
notes_to_parse = None
# grouping based on different instruments
parts = instrument.partitionByInstrument(midi)

if parts: # file has instrument parts
    notes_to_parse = parts.parts[0].recurse()
else: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

In [ ]:
unique_chars=sorted(list(set(notes)))     #no of pitches
print(unique_chars)

['0', '0.3', '0.4', '0.5', '1.3', '1.4', '1.5', '1.5.8', '1.6', '1.7', '10.0.4', '10.1', '10.1.5', '10.2', '10.2.5', '10.3', '2.4', '2.4.8', '2.5', '2.5.9', '2.6', '2.7', '3.6', '3.8', '3.9', '4', '4.10', '4.7', '4.9', '5', '5.10', '5.11', '5.7', '5.8', '5.9', '5.9.0', '7', '7.0', '7.10', '7.10.0', '7.10.1', '7.10.2', '8.0', '8.1', '8.11', '8.11.1', '9.0', '9.0.4', '9.1', '9.1.4', '9.2', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'B-3', 'B-4', 'B-5', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'C#2', 'C#3', 'C#4', 'C#5', 'C#6', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'E-3', 'E-4', 'E-5', 'E-6', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'F#3', 'F#4', 'F#5', 'F2', 'F3', 'F4', 'F5', 'F6', 'G#3', 'G#4', 'G#5', 'G#6', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6']


In [ ]:
n_vocab = len(unique_chars)
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [ ]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(network_output.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=50, batch_size=50, callbacks=callbacks_list)

Epoch 1/50
45/45 [==============================] - 131s 3s/step - loss: 4.3660 - accuracy: 0.0328
Epoch 2/50
45/45 [==============================] - 131s 3s/step - loss: 4.2380 - accuracy: 0.0391
Epoch 3/50
45/45 [==============================] - 130s 3s/step - loss: 4.2045 - accuracy: 0.0450
Epoch 4/50
45/45 [==============================] - 135s 3s/step - loss: 4.1753 - accuracy: 0.0441
Epoch 5/50
45/45 [==============================] - 130s 3s/step - loss: 4.0946 - accuracy: 0.0454
Epoch 6/50
45/45 [==============================] - 135s 3s/step - loss: 4.0591 - accuracy: 0.0513
Epoch 7/50
45/45 [==============================] - 134s 3s/step - loss: 4.0535 - accuracy: 0.0540
Epoch 8/50
45/45 [==============================] - 137s 3s/step - loss: 3.9573 - accuracy: 0.0535
Epoch 9/50
45/45 [==============================] - 134s 3s/step - loss: 3.9429 - accuracy: 0.0571
Epoch 10/50
45/45 [==============================] - 136s 3s/step - loss: 3.8353 - accuracy: 0.0683
Epoch 11/

In [ ]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 200 notes
for note_index in range(200):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    ind = numpy.asarray(index)
    pattern = numpy.append(pattern,ind)
    pattern = pattern[1:len(pattern)]

In [ ]:
print(prediction_output)

['F3', 'G6', 'F6', 'F6', 'E5', 'C5', 'D5', 'C5', 'A5', 'G5', 'F5', 'D5', 'C5', 'A4', 'G4', 'A4', 'G4', 'F4', 'D4', 'C4', 'A3', 'G3', 'A3', 'G3', 'F3', 'C3', 'A2', 'F2', '10.1.5', '10.1.5', 'F5', '5.9.0', '5.9.0', '5.9.0', '0.5', 'F3', '5', '5', 'F3', 'A6', 'F5', 'D6', 'D6', 'A4', 'A4', 'A4', 'F5', 'F5', '0', 'D5', '4.9', '5.9.0', '9.0', '5.9.0', 'G4', '5', '2.5', 'G4', 'E5', 'F2', 'E5', '9.0', 'A4', 'A4', 'D5', 'C3', 'E5', 'E5', '5.9', 'C4', 'B3', 'A4', 'D5', 'D5', 'E5', 'D4', '9.0', 'C4', 'A4', 'D5', 'F3', 'E5', '9.0', 'B3', 'C4', 'A4', 'D5', 'C3', 'E5', 'D4', 'A4', 'G5', 'C4', 'F5', 'B3', 'A4', 'D5', 'C4', 'E5', 'D4', '9.0', 'C4', 'C2', '10.2.5', 'G3', 'C4', '10.2.5', 'G4', 'C5', '10.2.5', 'G5', 'C6', '5.10', '2.7', '5.10', '2.7', '5.10', 'G5', '5.10', '2.7', '5.10', 'G5', '5.10', '2.7', 'C2', '10.2.5', 'G3', 'C4', '10.2.5', 'G4', 'C5', '10.2.5', 'G5', 'C6', '7.10.2', '0.5', '7.10.2', '0.5', '7.10', '0.5', '7.10.2', '0.5', '7.10.2', '0.5', '7.10', '0.5', '7.10.0', '0.4', '5.9', '0.5'

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output.mid')

'output.mid'